In [ ]:
'''quellen: https://github.com/lucidrains/denoising-diffusion-pytorch
            https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/annotated_diffusion.ipynb#scrollTo=3a159023
            https://deepfindr.github.io/
'''





import torch
import torchvision
import matplotlib.pyplot as plt

def show_images(datset, num_samples=20, cols=4):
    plt.figure(figsize=(15,15))
    for i, img in enumerate(data):
        if i == num_samples:
            break
        plt.subplot(int(num_samples/cols) + 1, cols, i + 1)
        plt.imshow(img[0])

data = torchvision.datasets.StanfordCars(root=".", download=True)
show_images(data)

In [ ]:
#forward process

import torch.nn.functional as F

# Lineares Beta-Schedule für den Diffusionsprozess
def linear_beta_schedule(timesteps, start=0.0001, end=0.02):
    return torch.linspace(start, end, timesteps)

# Hilfsfunktion, um einen Wert aus einer Liste basierend auf einem Index zu extrahieren
def get_index_from_list(vals, t, x_shape):
    batch_size = t.shape[0]
    out = vals.gather(-1, t.cpu())
    return out.reshape(batch_size, *((1,) * (len(x_shape) - 1))).to(t.device)

# Vorwärtsdiffusionsschritt: Hinzufügen von Rauschen zu den Eingabedaten
def forward_diffusion_sample(x_0, t, device="cpu"):
    noise = torch.randn_like(x_0)
    sqrt_alphas_cumprod_t = get_index_from_list(sqrt_alphas_cumprod, t, x_0.shape)
    sqrt_one_minus_alphas_cumprod_t = get_index_from_list(
        sqrt_one_minus_alphas_cumprod, t, x_0.shape
    )
    # Mittelwert + Varianz
    return sqrt_alphas_cumprod_t.to(device) * x_0.to(device) \
    + sqrt_one_minus_alphas_cumprod_t.to(device) * noise.to(device), noise.to(device)


# Definiere Beta-Schedule
T = 300
betas = linear_beta_schedule(timesteps=T)

# Vorausberechnete Werte für den geschlossenen Ausdruck (für die Beschleunigung des Diffusionsprozesses)
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, axis=0)
alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)
sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)
posterior_variance = betas * (1. - alphas_cumprod_prev) / (1. - alphas_cumprod)

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt

IMG_SIZE = 64
BATCH_SIZE = 128

# Funktion zum Laden und Transformieren des Datensatzes
def load_transformed_dataset():

    data_transforms = [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Lambda(lambda t: (t * 2) - 1)
    ]
    data_transform = transforms.Compose(data_transforms)

    #StanfordCars-Datensatz
    train = torchvision.datasets.StanfordCars(root=".", download=True,
                                         transform=data_transform)


    test = torchvision.datasets.StanfordCars(root=".", download=True,
                                         transform=data_transform, split='test')
    return torch.utils.data.ConcatDataset([train, test])


def show_tensor_image(image):
    # Definiere Transformationen, um das Bild für die Anzeige zurückzutransformieren
    reverse_transforms = transforms.Compose([
        transforms.Lambda(lambda t: (t + 1) / 2),
        transforms.Lambda(lambda t: t.permute(1, 2, 0)),
        transforms.Lambda(lambda t: t * 255.),
        transforms.Lambda(lambda t: t.numpy().astype(np.uint8)),
        transforms.ToPILImage(),
    ])


    # Wenn es ein Batch von Bildern ist, nehme nur das erste Bild
    if len(image.shape) == 4:
        image = image

data = load_transformed_dataset()
dataloader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [ ]:
from torch import nn
import math
import torch

# Definiert einen Block der Unet-Architektur (kann für Up- oder Downsampling verwendet werden)
class Block(nn.Module):
    def __init__(self, in_ch, out_ch, time_emb_dim, up=False):
        super().__init__()
        self.time_mlp = nn.Linear(time_emb_dim, out_ch)  # Zeit-Einbettungslayer
        if up:
            # Falls Upsampling: Verwendet eine Transponierte Faltung
            self.conv1 = nn.Conv2d(2*in_ch, out_ch, 3, padding=1)
            self.transform = nn.ConvTranspose2d(out_ch, out_ch, 4, 2, 1)
        else:
            # Falls Downsampling: Verwendet eine reguläre Faltung
            self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
            self.transform = nn.Conv2d(out_ch, out_ch, 4, 2, 1)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1)
        self.bnorm1 = nn.BatchNorm2d(out_ch)  # Batch Normalisierung
        self.bnorm2 = nn.BatchNorm2d(out_ch)
        self.relu = nn.ReLU()  # Aktivierungsfunktion ReLU

    def forward(self, x, t):
        # Erste Faltung und Batch-Normalisierung
        h = self.bnorm1(self.relu(self.conv1(x)))
        # Zeit-Einbettung anwenden
        time_emb = self.relu(self.time_mlp(t))
        # Die letzten 2 Dimensionen erweitern
        time_emb = time_emb[(..., ) + (None, ) * 2]
        # Zeitkanal hinzufügen
        h = h + time_emb
        # Zweite Faltung und Batch-Normalisierung
        h = self.bnorm2(self.relu(self.conv2(h)))
        # Down- oder Upsampling anwenden
        return self.transform(h)



class SinusoidalPositionEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)

        return embeddings


# Eine vereinfachte Version der Unet-Architektur
class SimpleUnet(nn.Module):
    def __init__(self):
        super().__init__()
        image_channels = 3

        down_channels = (64, 128, 256, 512, 1024)
        up_channels = (1024, 512, 256, 128, 64)
        out_dim = 3
        time_emb_dim = 32

        # Zeit-Einbettung
        self.time_mlp = nn.Sequential(
                SinusoidalPositionEmbeddings(time_emb_dim),
                nn.Linear(time_emb_dim, time_emb_dim),
                nn.ReLU()
            )

        # Initiale Projektion
        self.conv0 = nn.Conv2d(image_channels, down_channels[0], 3, padding=1)


        self.downs = nn.ModuleList([Block(down_channels[i], down_channels[i+1], \
                                    time_emb_dim) \
                    for i in range(len(down_channels)-1)])


        self.ups = nn.ModuleList([Block(up_channels[i], up_channels[i+1], \
                                        time_emb_dim, up=True) \
                    for i in range(len(up_channels)-1)])


        self.output = nn.Conv2d(up_channels[-1], out_dim, 1)

    def forward(self, x, timestep):

        t = self.time_mlp(timestep)

        x = self.conv0(x)
        # Unet: Downsampling und Speicherung der Zwischenwerte (Residuals)
        residual_inputs = []
        for down in self.downs:
            x = down(x, t)
            residual_inputs.append(x)
        # Upsampling und Hinzufügen der Residuals
        for up in self.ups:
            residual_x = residual_inputs.pop()

            x = torch.cat((x, residual_x), dim=1)
            x = up(x, t)
        return self.output(x)


model = SimpleUnet()
print("Num params: ", sum(p.numel() for p in model.parameters()))
model


In [ ]:
#loss

def get_loss(model, x_0, t):
    x_noisy, noise = forward_diffusion_sample(x_0, t, device)
    noise_pred = model(x_noisy, t)
    return F.l1_loss(noise, noise_pred)

In [ ]:
@torch.no_grad()
def sample_timestep(x, t):
    betas_t = get_index_from_list(betas, t, x.shape)
    sqrt_one_minus_alphas_cumprod_t = get_index_from_list(
        sqrt_one_minus_alphas_cumprod, t, x.shape
    )
    sqrt_recip_alphas_t = get_index_from_list(sqrt_recip_alphas, t, x.shape)


    model_mean = sqrt_recip_alphas_t * (
        x - betas_t * model(x, t) / sqrt_one_minus_alphas_cumprod_t
    )
    posterior_variance_t = get_index_from_list(posterior_variance, t, x.shape)

    if t == 0:
        return model_mean
    else:
        noise = torch.randn_like(x)
        return model_mean + torch.sqrt(posterior_variance_t) * noise


@torch.no_grad()
def sample_plot_image():


    img_size = IMG_SIZE
    img = torch.randn((1, 3, img_size, img_size), device=device)  # Zufälliges Bildrauschen
    plt.figure(figsize=(15,15))
    plt.axis('off')  # Achsen deaktivieren
    num_images = 10  # Anzahl der Bilder, die während des Samplings angezeigt werden
    stepsize = int(T/num_images)  # Schrittgröße für das Samplen

    for i in range(0, T)[::-1]:  # Iteriere von T bis 0
        t = torch.full((1,), i, device=device, dtype=torch.long)  # Erstelle Tensor mit aktuellem Zeitschritt t
        img = sample_timestep(img, t)  # Rufe den Sampling-Schritt auf
        img = torch.clamp(img, -1.0, 1.0)
        if i % stepsize == 0:
            plt.subplot(1, num_images, int(i/stepsize) + 1)  # Zeige das Bild in der entsprechenden Untergrafik an
            show_tensor_image(img.detach().cpu())  # Zeige das Tensorbild an
    plt.show()


In [ ]:
from torch.optim import Adam

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = Adam(model.parameters(), lr=0.001)
epochs = 300

for epoch in range(epochs):
    for step, batch in enumerate(dataloader):
      optimizer.zero_grad()

      t = torch.randint(0, T, (BATCH_SIZE,), device=device).long()
      loss = get_loss(model, batch[0], t)
      loss.backward()
      optimizer.step()

      if epoch % 5 == 0 and step == 0:
        print(f"Epoch {epoch} | step {step:03d} Loss: {loss.item()} ")
        sample_plot_image()